# Using AzureAIMemoryStore with LangGraph

This notebook demonstrates how to use the `AzureAIMemoryStore` class from `langchain-azure-ai` to provide **persistent, AI-powered long-term memory** to LangGraph agents.

Long-term memory allows agents to remember facts about users across conversations — for example, preferences, past interactions, or profile information. `AzureAIMemoryStore` implements LangGraph's `BaseStore` interface backed by Azure AI Foundry's memory stores service.

## What you will learn

1. How to create an `AzureAIMemoryStore` connected to Azure AI Foundry
2. How to use it with a **standard LangGraph agent** (`create_react_agent`)
3. How to use it with a **Microsoft Foundry agent** (`AgentServiceFactory`)

## Prerequisites

1. A Microsoft Foundry project (new Foundry experience)
2. A deployed chat model (e.g. `gpt-4.1`)
3. A **memory store** created in your Foundry project ([documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/memory-usage))
4. Environment variables set:
   - `AZURE_AI_PROJECT_ENDPOINT`
   - `MODEL_DEPLOYMENT_NAME`
   - `MEMORY_STORE_NAME`
5. Install dependencies:

```bash
pip install "langchain-azure-ai[preview]" "azure-ai-projects>=2.0.0b1" langchain-openai langgraph
```

## Setup

In [ ]:
import os

os.environ["AZURE_AI_PROJECT_ENDPOINT"] = "https://<resource>.services.ai.azure.com/api/projects/<project>"
os.environ["MODEL_DEPLOYMENT_NAME"] = "gpt-4.1"
os.environ["MEMORY_STORE_NAME"] = "my-memory-store"

If you run into issues, enable diagnostic logging:

In [ ]:
# Enable debug logging (optional)
import logging
logging.basicConfig(level=logging.WARNING)

### Helper functions

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage


def pretty_print(response: dict) -> None:
    """Print all messages in a LangGraph response."""
    for m in response["messages"]:
        m.pretty_print()

## Creating an AzureAIMemoryStore

`AzureAIMemoryStore` wraps Azure AI Foundry's memory stores API behind LangGraph's `BaseStore` interface. Under the hood it calls `search_memories`, `begin_update_memories`, and `delete_scope` to persist and retrieve memories.

You need an `AIProjectClient` (from the `azure-ai-projects` SDK V2) and the name of a memory store that already exists in your Foundry project.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_azure_ai.stores import AzureAIMemoryStore

credential = DefaultAzureCredential()

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential,
)

memory_store = AzureAIMemoryStore(
    project_client=project_client,
    memory_store_name=os.environ["MEMORY_STORE_NAME"],
)

print("AzureAIMemoryStore created successfully.")

### Direct store operations

Before plugging the store into an agent, let's verify it works by storing and searching memories directly. The `put` method feeds conversation content to the Azure AI memory service, which extracts and persists relevant facts. The `search` method retrieves memories that are relevant to a query.

In [ ]:
# Store a memory — the service will extract relevant facts from the content
memory_store.put(
    namespace=("user", "alice"),
    key="conv-1",
    value={"content": "My name is Alice and I love hiking in the mountains. I also enjoy Python programming."},
)

print("Memory stored for user alice.")

In [ ]:
# Search for memories relevant to a query
results = memory_store.search(
    ("user", "alice"),
    query="What are the user's hobbies?",
)

print(f"Found {len(results)} memories:")
for item in results:
    print(f"  [{item.key}] {item.value}")

## Example 1 — Standard LangGraph agent with AzureAIMemoryStore

In this example we use LangGraph's built-in `create_react_agent` with an Azure OpenAI chat model. The `store` parameter wires the `AzureAIMemoryStore` so that any node in the graph can read and write long-term memories.

The agent has access to a `recall_memories` tool that searches the store for memories relevant to the current conversation, and a `save_memory` tool that persists new information.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore
from langchain_core.tools import tool

# Create chat model pointing at an Azure OpenAI deployment
chat_model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    azure_ad_token_provider=credential.get_token("https://cognitiveservices.azure.com/.default").token,
    api_version="2025-04-01-preview",
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
)

In [ ]:
from langgraph.config import get_store


@tool
def recall_memories(query: str, user_id: str) -> str:
    """Search the memory store for information relevant to the query for a given user.

    Args:
        query: The search query describing what information to look for.
        user_id: The user whose memories to search.
    """
    store = get_store()
    results = store.search(("user", user_id), query=query, limit=5)
    if not results:
        return "No relevant memories found."
    return "\n".join(f"- {item.value.get('content', '')}" for item in results)


@tool
def save_memory(content: str, user_id: str) -> str:
    """Save a piece of information to the memory store for a given user.

    Args:
        content: The information to remember.
        user_id: The user to associate this memory with.
    """
    import uuid

    store = get_store()
    store.put(("user", user_id), str(uuid.uuid4()), {"content": content})
    return f"Memory saved for user {user_id}."

In [ ]:
# Create the agent with the memory store
langgraph_agent = create_react_agent(
    model=chat_model,
    tools=[recall_memories, save_memory],
    prompt="You are a helpful assistant with access to a long-term memory store. "
           "Use recall_memories to look up information about the user before answering. "
           "Use save_memory to remember important facts the user shares.",
    checkpointer=MemorySaver(),
    store=memory_store,
)

print("LangGraph agent created with AzureAIMemoryStore.")

Let's have a conversation with the agent. We already stored Alice's preferences earlier, so the agent should be able to recall them:

In [ ]:
config = {"configurable": {"thread_id": "session-1"}}

response = langgraph_agent.invoke(
    {"messages": [HumanMessage(content="What do you know about Alice's hobbies? My user_id is alice.")]},
    config=config,
)

pretty_print(response)

Now let's tell the agent something new and verify it gets persisted:

In [ ]:
response = langgraph_agent.invoke(
    {"messages": [HumanMessage(content="Please remember that Alice recently started learning Spanish. My user_id is alice.")]},
    config=config,
)

pretty_print(response)

In [ ]:
# In a new session, the agent should recall all of Alice's information
config_new = {"configurable": {"thread_id": "session-2"}}

response = langgraph_agent.invoke(
    {"messages": [HumanMessage(content="What do you know about Alice? My user_id is alice.")]},
    config=config_new,
)

pretty_print(response)

## Example 2 — Microsoft Foundry Agent with AzureAIMemoryStore

In this example we use `AgentServiceFactory` from `langchain-azure-ai` to create a Foundry-backed agent. The `store` parameter passes our `AzureAIMemoryStore` to the compiled graph, so Foundry agent nodes can also access long-term memory.

This pattern is useful when you want to combine Foundry's server-side agent execution with persistent, cross-session memory.

In [ ]:
from langchain_azure_ai.agents.v2 import AgentServiceFactory

factory = AgentServiceFactory(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential,
)

print("AgentServiceFactory created.")

### Creating the Foundry agent with tools and memory

We define local tools for memory recall and saving, then create a Foundry agent that can use them. The `store` parameter connects our `AzureAIMemoryStore`.

In [ ]:
foundry_agent = factory.create_prompt_agent(
    name="memory-demo-agent",
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    instructions=(
        "You are a helpful assistant with access to a long-term memory store. "
        "Use recall_memories to look up information about the user before answering. "
        "Use save_memory to remember important facts the user shares."
    ),
    tools=[recall_memories, save_memory],
    checkpointer=MemorySaver(),
    store=memory_store,
)

print(f"Foundry agent created: {factory.get_agents_id_from_graph(foundry_agent)}")

Let's visualise the agent graph:

In [ ]:
from IPython import display

display.Image(foundry_agent.get_graph().draw_mermaid_png())

### Conversing with the Foundry agent

Since we already stored Alice's preferences, the Foundry agent should be able to recall them too — both agents share the same memory store.

In [ ]:
config = {"configurable": {"thread_id": "foundry-session-1"}}

response = foundry_agent.invoke(
    {"messages": [HumanMessage(content="What do you remember about Alice? My user_id is alice.")]},
    config=config,
)

pretty_print(response)

Save new information via the Foundry agent:

In [ ]:
response = foundry_agent.invoke(
    {"messages": [HumanMessage(content="Please remember that Alice prefers dark mode in all her apps. My user_id is alice.")]},
    config=config,
)

pretty_print(response)

Verify the new memory is accessible from either agent — let's go back to the standard LangGraph agent:

In [ ]:
config_cross = {"configurable": {"thread_id": "session-cross-check"}}

response = langgraph_agent.invoke(
    {"messages": [HumanMessage(content="Tell me everything you know about Alice. My user_id is alice.")]},
    config=config_cross,
)

pretty_print(response)

## Cleanup

Delete the Foundry agent and optionally remove the memories stored for Alice.

In [ ]:
# Delete the Foundry agent from Azure AI Foundry
factory.delete_agent(foundry_agent)
print("Foundry agent deleted.")

In [ ]:
# Remove all memories for user alice
memory_store.delete(("user", "alice"), key="*")
print("Memories cleaned up.")

## Summary

In this notebook you learned how to:

- Create an `AzureAIMemoryStore` backed by Azure AI Foundry's memory stores service
- Use it directly to store and search memories
- Plug it into a **standard LangGraph agent** (`create_react_agent`) for long-term memory
- Plug it into a **Microsoft Foundry agent** (`AgentServiceFactory.create_prompt_agent`) for long-term memory
- Share the same memory store across different agent types

The `AzureAIMemoryStore` implements LangGraph's `BaseStore` interface, so it can be used as a drop-in replacement for `InMemoryStore` in any LangGraph application that needs persistent, AI-powered memory.